# Unitree G1 Humanoid Robot SDK Tutorial

This notebook provides a comprehensive guide to using the Unitree SDK2 Python interface for controlling the G1 humanoid robot.

## Table of Contents
1. [Setup & Installation](#setup)
2. [Audio Control](#audio)
3. [High-Level Control - Locomotion](#locomotion)
4. [High-Level Control - Arm Actions](#arm-actions)
5. [High-Level Control - Arm SDK (5-DOF)](#arm5)
6. [High-Level Control - Arm SDK (7-DOF)](#arm7)
7. [Low-Level Motor Control](#low-level)
8. [Data Visualization](#visualization)

---

## Resources & Documentation

- **GitHub Repository**: [unitree_sdk2_python](https://github.com/unitreerobotics/unitree_sdk2_python)
- **Official Documentation**: [Unitree Developer Portal](https://support.unitree.com/home/en/developer)
- **Robot Models**: G1 / H1-2 (uses `unitree_hg` IDL)
- **Python Version**: >= 3.8

---

<a id='setup'></a>
## 1. Setup & Installation

### Prerequisites
- Python >= 3.8
- Network connection to the robot
- Network interface name (e.g., `eth0`, `enp2s0`, `wlan0`)

### Installation Steps

In [ ]:
# Install uv package manager (fast pip replacement)
!python -m pip install uv

In [ ]:
# Create virtual environment with uv
!uv venv .venv

### Activate Virtual Environment

**Windows:**
```bash
.venv\Scripts\activate
```

**Linux/macOS:**
```bash
source .venv/bin/activate
```

In [ ]:
# Install core dependencies
!uv pip install cyclonedds==0.10.2 numpy opencv-python

In [ ]:
# Install Unitree SDK2 Python (editable mode)
!uv pip install -e unitree_sdk2_python

In [ ]:
# Install additional packages for data analysis and visualization
!uv pip install matplotlib pandas scipy seaborn pyrealsense2 open3d jupyter ipywidgets

### Troubleshooting: CycloneDDS Installation

If you encounter errors about `CYCLONEDDS_HOME` not being found, you may need to build CycloneDDS from source:

```bash
cd ~
git clone https://github.com/eclipse-cyclonedds/cyclonedds -b releases/0.10.x
cd cyclonedds && mkdir build install && cd build
cmake .. -DCMAKE_INSTALL_PREFIX=../install
cmake --build . --target install

# Set environment variable
export CYCLONEDDS_HOME="~/cyclonedds/install"

# Then retry SDK installation
cd ~/unitree_sdk2_python
pip3 install -e .
```

For details, see: https://pypi.org/project/cyclonedds/#installing-with-pre-built-binaries

### Network Configuration

Before running examples, configure network connection to the robot. See:
https://support.unitree.com/home/en/developer/Quick_start

**Find your network interface:**

In [ ]:
# Linux/macOS
!ip link show
# or
!ifconfig

In [ ]:
# Windows
!ipconfig

In [ ]:
# Set your network interface name
NETWORK_INTERFACE = "eth0"  # Change this to your actual interface (e.g., enp2s0, wlan0, etc.)

### Import Required Libraries

In [ ]:
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from dataclasses import dataclass

# Unitree SDK imports
from unitree_sdk2py.core.channel import ChannelSubscriber, ChannelPublisher, ChannelFactoryInitialize
from unitree_sdk2py.idl.unitree_hg.msg.dds_ import LowCmd_, LowState_
from unitree_sdk2py.utils.crc import CRC
from unitree_sdk2py.utils.thread import RecurrentThread

# G1-specific imports
from unitree_sdk2py.g1.audio.g1_audio_client import AudioClient
from unitree_sdk2py.g1.loco.g1_loco_client import LocoClient
from unitree_sdk2py.g1.arm.g1_arm_action_client import G1ArmActionClient, action_map
from unitree_sdk2py.comm.motion_switcher.motion_switcher_client import MotionSwitcherClient

print("✓ All imports successful!")

---
<a id='audio'></a>
## 2. Audio Control

Control the robot's audio output, text-to-speech, LED lights, and volume.

### Features:
- Volume control
- Text-to-Speech (TTS) in Chinese
- LED light control (RGB)
- WAV file playback (16kHz mono)

**Reference**: `example/g1/audio/g1_audio_client_example.py`

### 2.1 Basic Audio Control Example

In [ ]:
# Initialize DDS communication
ChannelFactoryInitialize(0, NETWORK_INTERFACE)

# Create audio client
audio_client = AudioClient()
audio_client.SetTimeout(10.0)
audio_client.Init()

# Create locomotion client (for wave hand demo)
sport_client = LocoClient()
sport_client.SetTimeout(10.0)
sport_client.Init()

print("✓ Audio client initialized")

In [ ]:
# Get current volume
current_volume = audio_client.GetVolume()
print(f"Current volume: {current_volume}")

In [ ]:
# Set volume (0-100)
audio_client.SetVolume(85)
print(f"Volume set to: {audio_client.GetVolume()}")

In [ ]:
# Wave hand gesture
sport_client.WaveHand()
time.sleep(2)

In [ ]:
# Text-to-Speech (Chinese)
audio_client.TtsMaker("大家好!我是宇树科技人形机器人。", 0)
time.sleep(5)

### 2.2 LED Control

In [ ]:
# LED Control - Red, Green, Blue (RGB values 0-255)
audio_client.TtsMaker("红色", 0)
audio_client.LedControl(255, 0, 0)  # Red
time.sleep(1)

audio_client.TtsMaker("绿色", 0)
audio_client.LedControl(0, 255, 0)  # Green
time.sleep(1)

audio_client.TtsMaker("蓝色", 0)
audio_client.LedControl(0, 0, 255)  # Blue
time.sleep(1)

### 2.3 Play WAV File

**Requirements**: 16kHz, Mono, PCM format

**Reference**: `example/g1/audio/g1_audio_client_play_wav.py`

In [ ]:
# Example WAV playback (requires wav.py helper functions)
# Uncomment and modify with your WAV file path

# from wav import read_wav, play_pcm_stream
# 
# wav_path = "path/to/your/audio.wav"
# pcm_list, sample_rate, num_channels, is_ok = read_wav(wav_path)
# 
# if is_ok and sample_rate == 16000 and num_channels == 1:
#     play_pcm_stream(audio_client, pcm_list, "example")
#     time.sleep(5)
#     audio_client.PlayStop("example")
# else:
#     print("Error: WAV must be 16kHz mono format")

---
<a id='locomotion'></a>
## 3. High-Level Control - Locomotion

Control robot movement, posture, and special motions.

### Available Commands:
- **Basic**: Damp, Stand Up/Down, Squat
- **Movement**: Forward, Lateral, Rotation
- **Posture**: Low Stand, High Stand, Zero Torque
- **Special**: Wave Hand, Shake Hand

**Reference**: `example/g1/high_level/g1_loco_client_example.py`

**Documentation**: https://support.unitree.com/home/en/developer/sports_services

In [ ]:
# Initialize locomotion client
ChannelFactoryInitialize(0, NETWORK_INTERFACE)

sport_client = LocoClient()
sport_client.SetTimeout(10.0)
sport_client.Init()

print("✓ Locomotion client initialized")
print("⚠️  WARNING: Ensure clear space around the robot!")

### 3.1 Available Motion Commands

In [ ]:
# Define available test options
@dataclass
class TestOption:
    name: str
    id: int
    description: str

locomotion_options = [
    TestOption("damp", 0, "Damping mode - motors relaxed"),
    TestOption("Squat2StandUp", 1, "Stand up from squatting position"),
    TestOption("StandUp2Squat", 2, "Squat down from standing"),
    TestOption("move forward", 3, "Move forward at 0.3 m/s"),
    TestOption("move lateral", 4, "Move sideways at 0.3 m/s"),
    TestOption("move rotate", 5, "Rotate at 0.3 rad/s"),
    TestOption("low stand", 6, "Lower standing posture"),
    TestOption("high stand", 7, "Higher standing posture"),
    TestOption("zero torque", 8, "Zero torque mode"),
    TestOption("wave hand1", 9, "Wave hand (no turn)"),
    TestOption("wave hand2", 10, "Wave hand with turn around"),
    TestOption("shake hand", 11, "Shake hand gesture"),
    TestOption("Lie2StandUp", 12, "Stand up from lying position"),
]

# Display options
print("Available Locomotion Commands:\n")
for opt in locomotion_options:
    print(f"  [{opt.id:2d}] {opt.name:20s} - {opt.description}")

### 3.2 Execute Locomotion Commands

In [ ]:
# Example: Damping mode
sport_client.Damp()
print("Executed: Damp")

In [ ]:
# Example: Stand up from squat
sport_client.Damp()
time.sleep(0.5)
sport_client.Squat2StandUp()
print("Executed: Squat to Stand Up")

In [ ]:
# Example: Move forward
sport_client.Move(0.3, 0, 0)  # (vx, vy, vyaw)
print("Executed: Move Forward (0.3 m/s)")
time.sleep(2)
sport_client.Move(0, 0, 0)  # Stop
print("Stopped")

In [ ]:
# Example: Wave hand without turning
sport_client.WaveHand()
print("Executed: Wave Hand")

In [ ]:
# Example: Wave hand with turn around
sport_client.WaveHand(True)
print("Executed: Wave Hand with Turn")

In [ ]:
# Example: Shake hand
sport_client.ShakeHand()
time.sleep(3)
sport_client.ShakeHand()  # Execute twice as per example
print("Executed: Shake Hand")

---
<a id='arm-actions'></a>
## 4. High-Level Control - Arm Actions

Pre-programmed arm gestures and social interactions.

### Available Actions:
- Social gestures (wave, shake hand, high five, hug)
- Expressive actions (heart, kiss, clap)
- Functional poses (hands up, reject)

**Reference**: `example/g1/high_level/g1_arm_action_example.py`

In [ ]:
# Initialize arm action client
ChannelFactoryInitialize(0, NETWORK_INTERFACE)

armAction_client = G1ArmActionClient()
armAction_client.SetTimeout(10.0)
armAction_client.Init()

print("✓ Arm action client initialized")
print("⚠️  WARNING: Ensure clear space around the robot!")

### 4.1 Available Arm Actions

In [ ]:
arm_action_options = [
    TestOption("release arm", 0, "Release arm control"),
    TestOption("shake hand", 1, "Handshake gesture"),
    TestOption("high five", 2, "High five gesture"),
    TestOption("hug", 3, "Hug gesture"),
    TestOption("high wave", 4, "High wave gesture"),
    TestOption("clap", 5, "Clapping motion"),
    TestOption("face wave", 6, "Wave near face"),
    TestOption("left kiss", 7, "Blow kiss (left hand)"),
    TestOption("heart", 8, "Heart shape with both hands"),
    TestOption("right heart", 9, "Heart shape (right emphasis)"),
    TestOption("hands up", 10, "Raise both hands"),
    TestOption("x-ray", 11, "X-ray pose (crossed arms)"),
    TestOption("right hand up", 12, "Raise right hand"),
    TestOption("reject", 13, "Rejection gesture"),
    TestOption("right kiss", 14, "Blow kiss (right hand)"),
    TestOption("two-hand kiss", 15, "Blow kiss (both hands)"),
]

# Display options
print("Available Arm Actions:\n")
for opt in arm_action_options:
    print(f"  [{opt.id:2d}] {opt.name:20s} - {opt.description}")

### 4.2 Execute Arm Actions

In [ ]:
# Example: Shake hand
armAction_client.ExecuteAction(action_map.get("shake hand"))
time.sleep(2)
armAction_client.ExecuteAction(action_map.get("release arm"))
print("Executed: Shake Hand")

In [ ]:
# Example: High five
armAction_client.ExecuteAction(action_map.get("high five"))
time.sleep(2)
armAction_client.ExecuteAction(action_map.get("release arm"))
print("Executed: High Five")

In [ ]:
# Example: Heart gesture
armAction_client.ExecuteAction(action_map.get("heart"))
time.sleep(2)
armAction_client.ExecuteAction(action_map.get("release arm"))
print("Executed: Heart Gesture")

In [ ]:
# Example: Clap
armAction_client.ExecuteAction(action_map.get("clap"))
time.sleep(3)
print("Executed: Clap")

---
<a id='arm5'></a>
## 5. High-Level Control - Arm SDK (5-DOF)

Direct control of 5-DOF arm joints (shoulder pitch/roll/yaw, elbow, wrist roll).

**Reference**: `example/g1/high_level/g1_arm5_sdk_dds_example.py`

**Note**: For G1 23-DOF variant (excludes wrist pitch/yaw)

### 5.1 Arm5 Joint Configuration

In [ ]:
# 5-DOF Arm joints (per arm):
# - LeftShoulderPitch, LeftShoulderRoll, LeftShoulderYaw
# - LeftElbow, LeftWristRoll
# - RightShoulderPitch, RightShoulderRoll, RightShoulderYaw
# - RightElbow, RightWristRoll
# + WaistYaw, WaistRoll, WaistPitch

print("""5-DOF Arm Control:
- Target Position: Arms raised to sides (T-pose variation)
- Control Parameters: kp=60.0, kd=1.5
- Duration: 3 stages (3s each)
  1. Move to zero posture
  2. Lift arms up
  3. Return to zero
  4. Release arm SDK
""")

In [ ]:
# This example requires running as a standalone script
# See: unitree_sdk2_python/example/g1/high_level/g1_arm5_sdk_dds_example.py

# To run:
# !python unitree_sdk2_python/example/g1/high_level/g1_arm5_sdk_dds_example.py {NETWORK_INTERFACE}

---
<a id='arm7'></a>
## 6. High-Level Control - Arm SDK (7-DOF)

Direct control of 7-DOF arm joints (includes wrist pitch and yaw).

**Reference**: `example/g1/high_level/g1_arm7_sdk_dds_example.py`

**Note**: For G1 29-DOF variant (includes wrist pitch/yaw)

### 6.1 Arm7 Joint Configuration

In [ ]:
# 7-DOF Arm joints (per arm):
# - LeftShoulderPitch, LeftShoulderRoll, LeftShoulderYaw
# - LeftElbow
# - LeftWristRoll, LeftWristPitch, LeftWristYaw
# (Same for right arm)
# + WaistYaw, WaistRoll, WaistPitch

print("""7-DOF Arm Control:
- Additional wrist pitch/yaw control
- More dexterous manipulation capability
- Same control flow as 5-DOF variant
""")

In [ ]:
# This example requires running as a standalone script
# See: unitree_sdk2_python/example/g1/high_level/g1_arm7_sdk_dds_example.py

# To run:
# !python unitree_sdk2_python/example/g1/high_level/g1_arm7_sdk_dds_example.py {NETWORK_INTERFACE}

---
<a id='low-level'></a>
## 7. Low-Level Motor Control

Direct motor control with position, velocity, and torque commands.

### Features:
- 29 motor control (legs, waist, arms)
- PD control (Kp, Kd gains)
- Two ankle control modes: PR (Pitch/Roll) and AB (A/B joints)
- IMU state feedback

**Reference**: `example/g1/low_level/g1_low_level_example.py`

**Documentation**: https://support.unitree.com/home/en/developer/Basic_services

⚠️ **WARNING**: Low-level control bypasses safety checks. Use with caution!

### 7.1 G1 Joint Index Reference

In [ ]:
class G1JointIndex:
    # Left leg
    LeftHipPitch = 0
    LeftHipRoll = 1
    LeftHipYaw = 2
    LeftKnee = 3
    LeftAnklePitch = 4  # Also LeftAnkleB
    LeftAnkleRoll = 5   # Also LeftAnkleA
    
    # Right leg
    RightHipPitch = 6
    RightHipRoll = 7
    RightHipYaw = 8
    RightKnee = 9
    RightAnklePitch = 10  # Also RightAnkleB
    RightAnkleRoll = 11   # Also RightAnkleA
    
    # Waist (INVALID for locked waist on 23/29 DOF)
    WaistYaw = 12
    WaistRoll = 13    # WaistA
    WaistPitch = 14   # WaistB
    
    # Left arm
    LeftShoulderPitch = 15
    LeftShoulderRoll = 16
    LeftShoulderYaw = 17
    LeftElbow = 18
    LeftWristRoll = 19
    LeftWristPitch = 20   # INVALID for 23 DOF
    LeftWristYaw = 21     # INVALID for 23 DOF
    
    # Right arm
    RightShoulderPitch = 22
    RightShoulderRoll = 23
    RightShoulderYaw = 24
    RightElbow = 25
    RightWristRoll = 26
    RightWristPitch = 27  # INVALID for 23 DOF
    RightWristYaw = 28    # INVALID for 23 DOF

print("G1 has 29 motors total (23-DOF or 29-DOF variants)")

### 7.2 Default Control Gains

In [ ]:
# Default Kp (Position gains)
Kp = [
    60, 60, 60, 100, 40, 40,      # Left leg
    60, 60, 60, 100, 40, 40,      # Right leg
    60, 40, 40,                   # Waist
    40, 40, 40, 40, 40, 40, 40,   # Left arm
    40, 40, 40, 40, 40, 40, 40    # Right arm
]

# Default Kd (Velocity gains)
Kd = [
    1, 1, 1, 2, 1, 1,       # Left leg
    1, 1, 1, 2, 1, 1,       # Right leg
    1, 1, 1,                # Waist
    1, 1, 1, 1, 1, 1, 1,    # Left arm
    1, 1, 1, 1, 1, 1, 1     # Right arm
]

print(f"Kp gains: {len(Kp)} motors")
print(f"Kd gains: {len(Kd)} motors")

### 7.3 Low-Level Control Example

The example demonstrates:
1. **Stage 1** (0-3s): Move to zero posture
2. **Stage 2** (3-6s): Swing ankles using PR mode (Pitch/Roll)
3. **Stage 3** (6s+): Swing ankles using AB mode (A/B joints) + wrist rotation

In [ ]:
# This example requires running as a standalone script with background threads
# See: unitree_sdk2_python/example/g1/low_level/g1_low_level_example.py

# To run:
# !python unitree_sdk2_python/example/g1/low_level/g1_low_level_example.py {NETWORK_INTERFACE}

### 7.4 Reading Low-Level State

In [ ]:
# Example: Subscribe to low-level state for monitoring
# This provides IMU data, motor states, battery voltage, etc.

state_data = []

def LowStateHandler(msg: LowState_):
    """Callback for low-level state updates"""
    state_data.append({
        'timestamp': time.time(),
        'imu_rpy': msg.imu_state.rpy,
        'battery_voltage': msg.power_v,
        # Add more fields as needed
    })

# Initialize subscriber
# ChannelFactoryInitialize(0, NETWORK_INTERFACE)
# lowstate_subscriber = ChannelSubscriber("rt/lowstate", LowState_)
# lowstate_subscriber.Init(LowStateHandler, 10)

print("Low-level state subscriber ready (uncomment to activate)")

---
<a id='visualization'></a>
## 8. Data Visualization

Visualize robot sensor data and telemetry.

### 8.1 IMU Data Visualization (Placeholder)

In [ ]:
# Placeholder for IMU data plotting
# Requires collecting state_data from low-level state subscriber

# Generate example data
t = np.linspace(0, 10, 100)
roll = 0.1 * np.sin(2 * np.pi * 0.5 * t) + np.random.normal(0, 0.01, len(t))
pitch = 0.05 * np.cos(2 * np.pi * 0.3 * t) + np.random.normal(0, 0.01, len(t))
yaw = 0.15 * np.sin(2 * np.pi * 0.2 * t) + np.random.normal(0, 0.01, len(t))

fig, axes = plt.subplots(3, 1, figsize=(12, 8))
axes[0].plot(t, roll, 'r-', label='Roll')
axes[0].set_ylabel('Roll (rad)')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(t, pitch, 'g-', label='Pitch')
axes[1].set_ylabel('Pitch (rad)')
axes[1].legend()
axes[1].grid(True)

axes[2].plot(t, yaw, 'b-', label='Yaw')
axes[2].set_ylabel('Yaw (rad)')
axes[2].set_xlabel('Time (s)')
axes[2].legend()
axes[2].grid(True)

plt.suptitle('IMU Orientation (Roll-Pitch-Yaw)', fontsize=14)
plt.tight_layout()
plt.show()

print("📊 IMU Data Visualization (Example Data)")

### 8.2 Joint Angle Visualization (Placeholder)

In [ ]:
# Placeholder for joint angle trajectories

joint_names = ['Hip Pitch', 'Hip Roll', 'Hip Yaw', 'Knee', 'Ankle Pitch', 'Ankle Roll']
num_joints = len(joint_names)
t = np.linspace(0, 5, 100)

fig, ax = plt.subplots(figsize=(12, 6))

for i, joint in enumerate(joint_names):
    # Generate example sinusoidal trajectories
    phase = i * np.pi / 6
    trajectory = 0.3 * np.sin(2 * np.pi * 0.5 * t + phase)
    ax.plot(t, trajectory, label=joint, linewidth=2)

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Joint Angle (rad)', fontsize=12)
ax.set_title('Left Leg Joint Angles', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 Joint Angle Visualization (Example Data)")

### 8.3 Battery & Power Statistics (Placeholder)

In [ ]:
# Placeholder for battery voltage and power consumption

t = np.linspace(0, 60, 300)  # 1 minute of data
battery_voltage = 48.0 - 0.1 * (t / 60) + np.random.normal(0, 0.05, len(t))
current_draw = 5.0 + 2.0 * np.sin(2 * np.pi * 0.2 * t) + np.random.normal(0, 0.2, len(t))
power = battery_voltage * current_draw

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 8))

ax1.plot(t, battery_voltage, 'b-', linewidth=1.5)
ax1.set_ylabel('Voltage (V)', fontsize=11)
ax1.set_title('Battery Voltage', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.axhline(y=44.0, color='r', linestyle='--', label='Low Battery Threshold')
ax1.legend()

ax2.plot(t, current_draw, 'g-', linewidth=1.5)
ax2.set_ylabel('Current (A)', fontsize=11)
ax2.set_title('Current Draw', fontsize=12)
ax2.grid(True, alpha=0.3)

ax3.plot(t, power, 'orange', linewidth=1.5)
ax3.set_xlabel('Time (s)', fontsize=11)
ax3.set_ylabel('Power (W)', fontsize=11)
ax3.set_title('Power Consumption', fontsize=12)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Battery & Power Statistics (Example Data)")

### 8.4 RealSense Depth Map Visualization (Placeholder)

In [ ]:
# Placeholder for RealSense depth camera visualization
# Requires pyrealsense2 library and actual camera data

# Generate synthetic depth map
depth_map = np.random.rand(480, 640) * 5.0  # Random depth 0-5 meters

# Add some structure (object in center)
y, x = np.ogrid[-240:240, -320:320]
mask = x**2 + y**2 <= 100**2
depth_map[mask] = 2.0  # Object at 2m

fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(depth_map, cmap='jet', vmin=0, vmax=5)
ax.set_title('RealSense Depth Map (Synthetic)', fontsize=14)
ax.set_xlabel('X (pixels)')
ax.set_ylabel('Y (pixels)')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Depth (m)', fontsize=11)
plt.tight_layout()
plt.show()

print("📊 RealSense Depth Map (Synthetic Data)")
print("To use real RealSense data, install pyrealsense2 and capture frames from camera")

### 8.5 LiDAR Point Cloud Visualization (Placeholder)

In [ ]:
# Placeholder for LiDAR point cloud visualization
# For full 3D visualization, use open3d library

# Generate synthetic LiDAR scan (2D slice)
num_points = 360
angles = np.linspace(0, 2*np.pi, num_points)
ranges = 3.0 + 0.5*np.sin(5*angles) + np.random.normal(0, 0.1, num_points)

# Convert to Cartesian coordinates
x = ranges * np.cos(angles)
y = ranges * np.sin(angles)

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))
ax.scatter(angles, ranges, c=ranges, cmap='viridis', s=10)
ax.set_title('LiDAR Scan (2D Slice)', fontsize=14, pad=20)
ax.set_ylim(0, 5)
plt.tight_layout()
plt.show()

# Cartesian plot
fig, ax = plt.subplots(figsize=(8, 8))
scatter = ax.scatter(x, y, c=ranges, cmap='viridis', s=10)
ax.scatter(0, 0, c='red', s=100, marker='x', label='Robot')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('LiDAR Point Cloud (Top-Down View)', fontsize=14)
ax.axis('equal')
ax.grid(True, alpha=0.3)
ax.legend()
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Range (m)', fontsize=11)
plt.tight_layout()
plt.show()

print("📊 LiDAR Point Cloud (Synthetic Data)")
print("For 3D visualization with real data, use open3d library")

---
## Summary

This notebook covered:

1. **SDK Installation** - Setting up unitree_sdk2_python with dependencies
2. **Audio Control** - TTS, volume, LED control, WAV playback
3. **Locomotion** - Walking, postures, special motions
4. **Arm Actions** - Pre-programmed gestures and social interactions
5. **Arm SDK** - Direct 5-DOF and 7-DOF arm control
6. **Low-Level Control** - Direct motor control with PD gains
7. **Visualization** - IMU, joints, battery, depth maps, LiDAR

### Next Steps:
- Integrate with ROS for navigation and planning
- Implement computer vision pipelines (OpenCV, RealSense)
- Develop custom behaviors and state machines
- Create teleoperation interfaces
- Add reinforcement learning for advanced control

### Additional Resources:
- [Unitree Developer Portal](https://support.unitree.com/home/en/developer)
- [SDK GitHub Issues](https://github.com/unitreerobotics/unitree_sdk2_python/issues)
- [Community Forum](https://www.unitree.com/community/)

---

**⚠️ Safety Reminders:**
- Always ensure clear space around the robot
- Start with damping mode before executing motions
- Use emergency stop if available
- Monitor battery levels
- Test in simulation first when possible

---

*Created with Unitree SDK2 Python v1.0.1*  
*For G1/H1-2 humanoid robots*